In [1]:
data_location = "/mnt/NewHDD"

In [2]:
import tensorflow as tf
import numpy as np
import InputList
import BinaryReader
import TFRecordsHandler
import time
import logging
import os
from importlib import reload
# reload(TFRecordsHandler)
# logging.basicConfig(filename='benchmark.log', encoding='utf-8', level=logging.DEBUG)

In [174]:
logging.info("Benchmark Start")

In [ ]:
br = BinaryReader.BinaryReader()
bag = br.create_test_dataset([InputList.diabetic_testing_files[0]])

In [4]:
def timeing(func):
    def inner(*args, **kwargs):
        start = time.time()
        logging.info(f"Starting with {func.__name__}")
        func(*args, **kwargs)
        end = time.time()
        logging.info(f"Finished with {func.__name__} in {end - start}")
    return inner

def instance_to_file(tensor, filename):
    with open(filename, 'wb') as f:
        np.save(f, tensor[0].numpy())

def file_to_instance(file):
    with open(file, 'rb') as f:
        a = np.load(f)
    return a, int(os.path.basename(file).decode('ascii')[0])

def numpy_instance_generator(instance_file_list):
    for file in instance_file_list:
        yield file_to_instance(file)

In [ ]:
def tfrecords_real_writer():
    br = BinaryReader.BinaryReader()
    for file in file_list:
        bag = br.create_test_dataset([file])
        i = 0
        dirname = os.path.join(data_location, "tfrecords")
        if not os.path.exists(dirname):
            os.mkdir(dirname)
        for instance in bag:
            filename = os.path.join(dirname, f"{os.path.basename(file[0])}_{i}")
            TFRecordsHandler.write_images_to_tfr_short(*instance, filename=filename)
            i += 1

('/mnt/p_Zeiss/Projects/UWF OCTA/Clinical data/MOON1/D39/links',
 'raw_1536x2048x2045x2_31711.bin')

In [50]:
@timeing
def tfrecords_writer():
    i = 0
    dirname = os.path.join(data_location, "tfrecords")
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    for instance in bag:
        filename = os.path.join(dirname, f"file_{i}")
        TFRecordsHandler.write_images_to_tfr_short(*instance, filename=filename)
        i += 1

@timeing
def numpy_writer():
    i = 0
    dirname = os.path.join(data_location, "numpy")
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    for instance in bag:
        filename = os.path.join(dirname, f"{instance[1]}_file_{i}.npy")
        instance_to_file(instance, filename)
        i += 1

def tfrecords_reader():
    dir_location = os.path.join(data_location, "tfrecords")
    list_of_files = os.listdir(dir_location)
    list_of_file_paths = list(map(lambda x: os.path.join(dir_location, x), list_of_files))
    dataset = tf.data.Dataset.from_tensor_slices(list_of_file_paths)
    dataset = dataset.interleave(lambda x: tf.data.TFRecordDataset(x),
                                 num_parallel_calls=tf.data.AUTOTUNE, cycle_length=4,
                                 deterministic=False) \
                      .prefetch(tf.data.AUTOTUNE)

    # pass every single feature through our mapping function
    dataset = dataset.map(
      TFRecordsHandler.parse_tfr_element
    )
    dataset.batch(4)
    return dataset

def numpy_reader():
    dir_location = os.path.join(data_location, "numpy")
    list_of_files = os.listdir(dir_location)
    list_of_file_paths = list(map(lambda x: os.path.join(dir_location, x), list_of_files))
    dataset = tf.data.Dataset.from_tensor_slices(list_of_file_paths) \
                .interleave(lambda x:
        tf.data.Dataset.from_generator(numpy_instance_generator, args=[list_of_file_paths],
                                             output_signature=(
                                                tf.TensorSpec(
                                                    shape=(1536, 102, 102, 1),
                                                    dtype=np.dtype('u2')),
                                                tf.TensorSpec(
                                                    shape=(),
                                                    dtype=np.dtype('u1')))
                                             ),
                            num_parallel_calls=tf.data.AUTOTUNE,
                            deterministic=True)\
        .prefetch(tf.data.AUTOTUNE)
    return dataset

def numpy_reader2():
    dir_location = os.path.join(data_location, "numpy")
    list_of_files = os.listdir(dir_location)
    list_of_file_paths = list(map(lambda x: os.path.join(dir_location, x), list_of_files))
    dataset = tf.data.Dataset.from_generator(numpy_instance_generator, args=[list_of_file_paths],
                                             output_signature=(
                                                tf.TensorSpec(
                                                    shape=(1536, 102, 102, 1),
                                                    dtype=np.dtype('u2')),
                                                tf.TensorSpec(
                                                    shape=(),
                                                    dtype=np.dtype('u1')))
                                             )
    return dataset.batch(4)


def classy_reader():
    return br.create_dataset([InputList.diabetic_testing_files[0]], True)

@timeing
def tfrecords_iterator(dataset):
    for sample in dataset:
        garbage = sample[0]
    print(garbage)

@timeing
def numpy_iterator(dataset):
    garbage = 0
    for sample in dataset:
        garbage += sample[1]
    print(garbage)

@timeing
def classy_iterator(dataset):
    garbage = 0
    for sample in dataset:
        garbage += sample[1]
    print(garbage)

In [18]:
tfrecords_writer()
#numpy_writer()

In [51]:
tfdataset = tfrecords_reader()

In [41]:
numpy_dataset = numpy_reader2()

In [48]:
classy_dataset = classy_reader()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [52]:
tfrecords_iterator(tfdataset)

tf.Tensor(
[[[[34016]
   [37664]
   [34784]
   ...
   [32864]
   [33056]
   [31136]]

  [[36032]
   [34304]
   [35712]
   ...
   [32896]
   [34816]
   [31488]]

  [[34656]
   [36160]
   [36096]
   ...
   [30720]
   [34912]
   [28992]]

  ...

  [[34016]
   [32512]
   [35744]
   ...
   [31712]
   [34400]
   [32352]]

  [[34784]
   [34816]
   [36864]
   ...
   [28832]
   [33888]
   [31168]]

  [[33824]
   [33568]
   [32992]
   ...
   [30624]
   [30176]
   [32128]]]


 [[[34816]
   [38048]
   [35136]
   ...
   [35328]
   [30784]
   [29984]]

  [[35872]
   [35360]
   [37504]
   ...
   [31904]
   [33312]
   [30848]]

  [[34592]
   [36768]
   [34688]
   ...
   [30176]
   [33952]
   [30144]]

  ...

  [[35200]
   [34592]
   [37696]
   ...
   [29792]
   [35232]
   [31712]]

  [[36608]
   [35168]
   [40160]
   ...
   [29952]
   [36512]
   [32512]]

  [[33408]
   [35200]
   [35296]
   ...
   [30368]
   [33504]
   [34176]]]


 [[[34272]
   [37824]
   [32864]
   ...
   [36896]
   [32672]
   [29472

In [44]:
numpy_iterator(numpy_dataset)

tf.Tensor([97 99 98 97], shape=(4,), dtype=uint8)


In [49]:
classy_iterator(classy_dataset)

tf.Tensor(135, shape=(), dtype=uint8)


In [ ]:
parsed_dataset = tf.data.TFRecordDataset(["images.tfrecords"])

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(parsed_record))

In [ ]:
from TFRecordsHandler import get_dataset_small

In [ ]:
instance_to_file(sample)

In [ ]:
file_list = ["0_file.npy"]
dataset = tf.data.Dataset.from_generator(numpy_instance_generator, args=[file_list],
                                         output_signature=(
                                            tf.TensorSpec(
                                                shape=(1536, 102, 102, 1),
                                                dtype=np.dtype('u2')),
                                            tf.TensorSpec(
                                                shape=(),
                                                dtype=np.dtype('u1')))
                                         )

In [ ]:
dataset